In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [ ]:
plant = pd.read_csv("plants_herbaldb_df", sep=";")
plant_compound_herbaldb = []
for plant in plant["id"]:
   url = requests.get('http://herbaldb.farmasi.ui.ac.id/v3/popdetails.php?speciesid='+plant)
      
   response = BeautifulSoup(url.content, 'html.parser')
   response_table = response.find_all("table")

   if len(response_table) >= 4:
      compound_table_html = response_table[4]

      # get tr
      compound_table_tr = compound_table_html.find_all("tr")
      if len(compound_table_tr) != 2:
         for i in range(len(compound_table_tr)):
            data_tmp = []
            data_tmp.append(plant)
            compound_table_td = compound_table_tr[i].find_all("td")
            for j in compound_table_td:
               if i != 0:
                  data_tmp.append(j.get_text())
            if i != 0:
               plant_compound_herbaldb.append(data_tmp)

# convert to dataframe
columns = ['plantId', 'no', 'compound', 'knapsackId', 'metaboliteId', 'pubchemId', 'reference', 'status', 'aksi']
plant_compound_herbaldb_df = pd.DataFrame(plant_compound_herbaldb, columns=columns)

# export plant - compound herbal db
plant_compound_herbaldb_df = plant_compound_herbaldb_df[['plantId', 'compound', 'knapsackId', 'metaboliteId', 'pubchemId', 'reference']]
plant_compound_herbaldb_df.to_csv("plant_compound_herbaldb.csv", sep=";")

# get compound data from plant compound herbaldb dataframe
compound_herbaldb_df = plant_compound_herbaldb_df[['compound', 'knapsackId', 'metaboliteId', 'pubchemId', 'reference']]
compound_herbaldb_df = compound_herbaldb_df.drop_duplicates()
compound_herbaldb_df.to_csv("compound_herbaldb.csv", sep=";")

print("\n==== SUMMARY PLANT-COUMPOUND ====")
print("\nHerbaldb Plant-Compound Pairs: ")
print(plant_compound_herbaldb_df.head())

print("\nHerbaldb Compound : ")
print(compound_herbaldb_df.head())
print("\n\nCount Herbaldb Plant Compound Pairs : ", len(plant_compound_herbaldb_df))
print("Count Herbaldb Compounds: ", len(compound_herbaldb_df))